In [ ]:
#Taller #6: Word2Vec¶


In [4]:
##Importamos librerías, cargamos el archivo que vamos a procesar
#hacemos tokenización y limpieza de texto (quitar palabras vacias, convertit a minusculas, quitar signos especiales, 
#creamos una nueva columna sobre una nueva colummna y visualizamos el dataframe)
import re
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto.split()

cuentos = pd.read_csv("archivos/colombia.csv", sep=";", encoding='utf-8')
cuentos['preproce'] = cuentos['Fragmento'].apply(lambda texto: pre_procesado(texto))
cuentos.head()

,Autor,nombre obra,Fragmento,preproce
0,Rafael Pombo,Simón el bobito\n,Simón el bobito llamó al pastelero:\n¡a ver lo...,"[simón, bobito, llamó, pastelero, ver, pastele..."
1,Rafael Pombo,La pobre viejecita\n,Érase una viejecita\nSin nadita que comer\nSin...,"[érase, viejecita, nadita, comer, sino, carnes..."
2,Rafael Pombo,El gato bandido\n,"Michín dijo a su mamá:\n""Voy a volverme Pateta...","[michín, dijo, mamá, voy, volverme, pateta, im..."
3,Rafael Pombo,Cutufato y su gato\n,Quiso el niño Cutufato\nDivertirse con un gato...,"[quiso, niño, cutufato, divertirse, gato, ató,..."
4,Rafael Pombo,De noche,No ya mi corazón desasosiegan\nlas mágicas vis...,"[corazón, desasosiegan, mágicas, visiones, día..."


In [5]:
#Creamos nuestro propio modelo, con nuestro dataset

In [6]:
import gensim.models.word2vec as w2v

In [7]:
%%time

mi_modelo = w2v.Word2Vec(cuentos['preproce'].values,
                            #sg=0, # 1 para usar skip-gram, 0 para usar CBOW
                            #seed=1, # 1 para tener resultados reproducibles y debugear
                            #size=512, # el tamaño de los vectores de palabras, o neuronas
                            #min_count=50, #mínimo de veces que ha de aparecer cada palabra, para ser considerada,
                            window=12) # ventana contextual de cada palabra




Wall time: 184 ms


In [8]:
#Instalamos la librería gensim
!pip install --user gensim 

In [11]:
#Contamos el número de palabras de nuestro vocabulario
len(mi_modelo.wv.vocab)
mi_modelo.wv.vocab


{'simón': <gensim.models.keyedvectors.Vocab at 0x213d1d144c8>,
 'ver': <gensim.models.keyedvectors.Vocab at 0x213d1d14548>,
 'quiero': <gensim.models.keyedvectors.Vocab at 0x213d1d14588>,
 'buen': <gensim.models.keyedvectors.Vocab at 0x213d1d145c8>,
 'dijo': <gensim.models.keyedvectors.Vocab at 0x213d1d14648>,
 'pescado': <gensim.models.keyedvectors.Vocab at 0x213cc4f1108>,
 'pasa': <gensim.models.keyedvectors.Vocab at 0x213cc4f15c8>,
 'sentado': <gensim.models.keyedvectors.Vocab at 0x213cc4fc888>,
 'mamá': <gensim.models.keyedvectors.Vocab at 0x213cc4fc108>,
 'hizo': <gensim.models.keyedvectors.Vocab at 0x213cc4f13c8>,
 'nieve': <gensim.models.keyedvectors.Vocab at 0x213cc461748>,
 'vio': <gensim.models.keyedvectors.Vocab at 0x213cc45e348>,
 'voy': <gensim.models.keyedvectors.Vocab at 0x213cc517748>,
 'puntas': <gensim.models.keyedvectors.Vocab at 0x213cc505188>,
 'negro': <gensim.models.keyedvectors.Vocab at 0x213cc505208>,
 'dio': <gensim.models.keyedvectors.Vocab at 0x213cc505348>,

In [26]:
mi_modelo.wv.most_similar("piedra")

[('si', 0.9178486466407776),
 ('muerte', 0.9078915119171143),
 ('dos', 0.9075802564620972),
 ('día', 0.9072862267494202),
 ('madre', 0.9068607091903687),
 ('año', 0.9027260541915894),
 ('hace', 0.9017083048820496),
 ('tierra', 0.9012608528137207),
 ('casa', 0.9011459946632385),
 ('aquí', 0.8987990617752075)]

In [15]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = mi_modelo.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [27]:
nearest_similarity_cosmul("llega", "llegaron", "tierra")

llega es a llegaron, lo que aire es a tierra


In [29]:
vocab = [e[0] for e in mi_modelo.wv.most_similar("piedra")]+[e[0] for e in mi_modelo.wv.most_similar("gato")]
print(vocab)

['si', 'muerte', 'dos', 'día', 'madre', 'año', 'hace', 'tierra', 'casa', 'aquí', 'castellano', 'luz', 'hombre', 'paso', 'tiempo', 'grandes', 'tarde', 'muerte', 'día', 'años']


In [ ]:
##¿Su modelo da buenos resultados? ¿Por qué sí o por qué no?
#Rta: Creo que no me dio los resultados que esperaba, ya que escogí un dataset de diferentes narraciones literarias colombianas (cuentos, poemas, fábulas, mitos y leyendas) y casi no tiene relación ya que tratan sobre temas diferentes

In [ ]:
##¿Qué problemas encontró al realizar este taller?
#Fue dificil encontrar un buen dataset, el que se escogió para este ejercicio fue una recopilación de varias cuentos colombianos, en total 49, pero al realizar la limpieza de la data, la misma sólo quedó con 340 palabras, lo cual no es bueno para realizar este tipo de análisis.
#Adicional, no fue tan fácil realizar cosas que no se aprendieron en la clase. 